# Strategy Tester proof of concept
- Author(s):
  - Joe Ang
- Description
  - Strategy testing using BackTrader with StocksCafe API.

## Dependent modules

In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import datetime
# Import the backtrader platform
import backtrader as bt
from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Tradimo
cus_module_path = '../'
sys.path.append(cus_module_path)
# Import Stocks Cafe API
from stockscafe.StocksCafeApi import StocksCafeApi
# Import Stocks Cafe helper functions
import lib

Loading BokehJS ...

## Define indicator and strategy classes

In [2]:
class BreakoutBox(bt.Indicator):
    alias = ('BBox')
    lines = ('top', 'bot')
    params = (
        # Window period for breakout
        ('breakout_period', 5),
        # How many dollars beyond period high/low to qualify for breakout.
        ('exceed_by', 0.1)
    )
    plotinfo = dict(subplot=False)
    
    def __init__(self):
        self.lines.top = bt.indicators.Highest(
                            self.data.high + self.p.exceed_by,
                            period = self.p.breakout_period)
        self.lines.bot = bt.indicators.Lowest(
                            self.data.low - self.p.exceed_by,
                            period = self.p.breakout_period)

# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        #print('{}, {}, {}'.format(self.datas[0]._name, dt.isoformat(), txt))
        print('{}, {}'.format(dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the high, low lines in the data[0] dataseries
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low
        self.dataclose = self.datas[0].close
        self.bbox = BreakoutBox()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    #def windowHigh(self, windows_size = 5):


    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f' % self.dataclose[0])
        self.log('H {:.2f}, T {:.2f}, L {:.2f}, B {:.2f}'.format(
                    self.datahigh[0], self.bbox.lines.top[0],
                    self.datalow[0], self.bbox.lines.bot[0])
        )
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        try:
            if self.order:
                return
        except AttributeError:
            pass

        # Check if we are in the market
        if not self.position:
            '''
            order = self.sell(exectype=bt.Order.StopLimit,
                        price=self.bbox.lines.bot,
                        plimit=self.bbox.lines.bot)
            '''
            order_validity = datetime.datetime.now()# + datetime.timedelta(days=1)
            self.order = self.buy(exectype=bt.Order.StopLimit,
                        price=self.bbox.lines.top,
                        plimit=self.bbox.lines.top,
                        valid = order_validity
            )

            # BUY, BUY, BUY!!! (with default parameters)
            self.log('BUY Stop Limit CREATE, %.2f' % self.bbox.lines.top[0])
        else:
            order_validity = datetime.datetime.now()
            # Already in the market ... we might sell
            self.order = self.sell(exectype=bt.Order.StopLimit,
                        price=self.bbox.lines.bot,
                        plimit=self.bbox.lines.bot,
                        valid = order_validity
            )
            self.log('SELL Stop Limit CREATE, %.2f' % self.bbox.lines.bot[0])



## Main code

In [3]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))

    # Get Stocks Cafe symbol prices
    symbols = [
        ('SGX', 'D05')
        #,('SGX', 'ME8U')
        #,('SGX', 'T82U')
    ]
    sc = StocksCafeApi(api_key_file='api-key.txt')
    prices = []
    for (exchange, symbol) in symbols:
        symbol_price = lib.get_symbol_prices(sc, exchange, symbol,
                                        start_date='2018-04-01', end_date=None)
        symbol_price = symbol_price[[ 'date',
                        'open', 'high', 'low', 'close', 'volume']]
        data = bt.feeds.PandasData(dataname=symbol_price,
                               datetime='date',
                               nocase=True)
        # Add the Stocks Cafe symbol prices to Cerebro
        cerebro.adddata(data, name = symbol)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)


### Log of indicator values and order executions

In [4]:
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # Run over everything
    cerebro.run()
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00
2018-04-06, H 27.78, T 27.88, L 27.00, B 26.25
2018-04-06, BUY Stop Limit CREATE, 27.88
2018-04-09, BUY EXECUTED, 27.88
2018-04-09, H 28.05, T 28.15, L 27.30, B 26.25
2018-04-09, SELL Stop Limit CREATE, 26.25
2018-04-10, H 28.05, T 28.15, L 27.58, B 26.25
2018-04-11, H 28.17, T 28.27, L 27.95, B 26.76
2018-04-12, H 28.30, T 28.40, L 28.00, B 26.90
2018-04-13, H 28.49, T 28.59, L 28.25, B 27.20
2018-04-16, H 28.60, T 28.70, L 28.28, B 27.48
2018-04-17, H 28.88, T 28.98, L 28.42, B 27.85
2018-04-18, H 29.45, T 29.55, L 28.99, B 27.90
2018-04-19, H 29.95, T 30.05, L 29.50, B 28.15
2018-04-20, H 29.95, T 30.05, L 29.45, B 28.18
2018-04-23, H 29.77, T 30.05, L 29.23, B 28.32
2018-04-24, H 30.00, T 30.10, L 29.48, B 28.89
2018-04-25, H 29.99, T 30.10, L 29.64, B 29.13
2018-04-26, H 29.99, T 30.10, L 29.70, B 29.13
2018-04-27, H 30.00, T 30.10, L 29.75, B 29.13
2018-04-30, H 30.98, T 31.08, L 30.43, B 29.38
2018-05-02, H 31.28, T 31.38, L 30.51, B 29.54
201

### Visualize the prices, indicators and orders

In [5]:
    # Plot the result
    b = Bokeh(style='bar', plot_mode='single', scheme=Tradimo())
    cerebro.plot(b)
    #cerebro.plot(style='candlestick')

[[<backtrader_plotting.bokeh.bokeh.FigurePage at 0x1a1d3342b0>]]